In [1]:
import pandas as pd
import numpy as np
import ftfy
import unicodedata
import re
from lxml import etree
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
train_labels = pd.read_csv('data/train_labels.csv')
train_labels.head()

,article_id,dataset_id,type
0,10.1002_2017jc013030,https://doi.org/10.17882/49388,Primary
1,10.1002_anie.201916483,Missing,Missing
2,10.1002_anie.202005531,Missing,Missing
3,10.1002_anie.202007717,Missing,Missing
4,10.1002_chem.201902131,Missing,Missing


In [3]:
train_labels[train_labels['dataset_id'] == 'SAMN07159041']

,article_id,dataset_id,type
848,10.3389_fmicb.2024.1456637,SAMN07159041,Secondary


In [4]:
train_labels[train_labels['article_id'] == '10.3389_fmicb.2024.1456637']

,article_id,dataset_id,type
847,10.3389_fmicb.2024.1456637,NC_017643.1,Secondary
848,10.3389_fmicb.2024.1456637,SAMN07159041,Secondary
849,10.3389_fmicb.2024.1456637,SAMN07159044,Secondary
850,10.3389_fmicb.2024.1456637,SAMN07159045,Secondary
851,10.3389_fmicb.2024.1456637,SAMN08172567,Secondary
852,10.3389_fmicb.2024.1456637,SAMN08199278,Secondary
853,10.3389_fmicb.2024.1456637,SAMN09534374,Secondary
854,10.3389_fmicb.2024.1456637,SAMN12620691,Secondary
855,10.3389_fmicb.2024.1456637,SAMN12620692,Secondary
856,10.3389_fmicb.2024.1456637,SAMN12620693,Secondary


In [5]:
train_labels.shape

(1028, 3)

In [6]:
train_labels['dataset_id'].isna().sum()

np.int64(0)

In [7]:
train_labels[train_labels['dataset_id'] == 'Missing'].shape

(309, 3)

In [8]:
train_labels['dataset_id'].nunique()

720

In [9]:
train_labels.loc[train_labels['dataset_id'].str.contains('https://doi.org/10'), 'dataset_id'] = 1
train_labels.loc[train_labels['dataset_id'] == 'Missing', 'dataset_id'] = 0
train_labels['dataset_id'].nunique()

396

In [10]:
print(list(train_labels['dataset_id'].unique()))


[1, 0, '5VA1', 'CHEMBL1097', 'CHEMBL1257578', 'CHEMBL1257821', 'CHEMBL1782574', 'CHEMBL195180', 'CHEMBL2424928', 'CHEMBL3422978', 'CHEMBL390649', 'CHEMBL572163', 'CHEMBL70', 'IPR000884', 'IPR001124', 'IPR001577', 'IPR002477', 'IPR002889', 'IPR004007', 'IPR004302', 'IPR004785', 'IPR004911', 'IPR004963', 'IPR004981', 'IPR005524', 'IPR006201', 'IPR008139', 'IPR009613', 'IPR011040', 'IPR011124', 'IPR012338', 'IPR016201', 'IPR017920', 'IPR021345', 'IPR028730', 'IPR029723', 'IPR029787', 'IPR033396', 'IPR036452', 'PF00348', 'PF00484', 'PF01493', 'PF05834', 'PF15461', 'GSE130386', 'SRP184152', 'EMPIAR-10005', 'EMPIAR-10017', 'EMPIAR-10028', 'EMPIAR-10081', 'EMPIAR-10093', 'EMPIAR-10345', 'EMPIAR-10387', 'EMPIAR-10406', 'EMPIAR-10532', 'EMPIAR-10590', 'EMPIAR-10852', 'EMPIAR-11057', 'ENSBTAG00000011038', 'ENSBTAG00000013718', 'ENSBTAG00000017121', 'ENSBTAG00000017131', 'ENSBTAG00000021275', 'ENSBTAG00000047833', 'ENSOARG00000003950', 'ENSOARG00000012128', 'ENSOARG00000012835', 'ENSOARG000000137

1028


In [11]:
len(list(train_labels['dataset_id'].unique()))

396

In [12]:
li = list(train_labels['dataset_id'].unique())
li = [item for item in li if item not in [0, 1]]



In [13]:
samn = [item for item in li if item.startswith('SAMN')]
print(len(samn))
print(samn)

41
['SAMN07159041', 'SAMN07159044', 'SAMN07159045', 'SAMN08172567', 'SAMN08199278', 'SAMN09534374', 'SAMN12620691', 'SAMN12620692', 'SAMN12620693', 'SAMN12620694', 'SAMN12620697', 'SAMN12620700', 'SAMN12620701', 'SAMN12620702', 'SAMN12620703', 'SAMN12620704', 'SAMN17525956', 'SAMN16233641', 'SAMN16233642', 'SAMN16233643', 'SAMN16233644', 'SAMN16233645', 'SAMN16233646', 'SAMN16233647', 'SAMN16233648', 'SAMN16233649', 'SAMN16233650', 'SAMN16233651', 'SAMN16233652', 'SAMN16233653', 'SAMN16233654', 'SAMN16233655', 'SAMN16233656', 'SAMN16233657', 'SAMN16233658', 'SAMN16233659', 'SAMN16233660', 'SAMN16233661', 'SAMN16233662', 'SAMN16233663', 'SAMN16233664']


In [14]:
samns = re.findall(r'\bSAMN\d{8}\b', ' '.join(li))
print(len(samns))
print(samns)

41
['SAMN07159041', 'SAMN07159044', 'SAMN07159045', 'SAMN08172567', 'SAMN08199278', 'SAMN09534374', 'SAMN12620691', 'SAMN12620692', 'SAMN12620693', 'SAMN12620694', 'SAMN12620697', 'SAMN12620700', 'SAMN12620701', 'SAMN12620702', 'SAMN12620703', 'SAMN12620704', 'SAMN17525956', 'SAMN16233641', 'SAMN16233642', 'SAMN16233643', 'SAMN16233644', 'SAMN16233645', 'SAMN16233646', 'SAMN16233647', 'SAMN16233648', 'SAMN16233649', 'SAMN16233650', 'SAMN16233651', 'SAMN16233652', 'SAMN16233653', 'SAMN16233654', 'SAMN16233655', 'SAMN16233656', 'SAMN16233657', 'SAMN16233658', 'SAMN16233659', 'SAMN16233660', 'SAMN16233661', 'SAMN16233662', 'SAMN16233663', 'SAMN16233664']


In [15]:
che = [item for item in li if item.startswith('CHEMBL')]
print(len( che))
print(che)

29
['CHEMBL1097', 'CHEMBL1257578', 'CHEMBL1257821', 'CHEMBL1782574', 'CHEMBL195180', 'CHEMBL2424928', 'CHEMBL3422978', 'CHEMBL390649', 'CHEMBL572163', 'CHEMBL70', 'CHEMBL1568820', 'CHEMBL1652119', 'CHEMBL198796', 'CHEMBL243644', 'CHEMBL2436978', 'CHEMBL3109593', 'CHEMBL3311228', 'CHEMBL3752642', 'CHEMBL3960662', 'CHEMBL402487', 'CHEMBL403', 'CHEMBL4065616', 'CHEMBL408500', 'CHEMBL561', 'CHEMBL64391', 'CHEMBL76', 'CHEMBL3347550', 'CHEMBL3347551', 'CHEMBL3733336']


In [16]:
import re
import ftfy
import unicodedata

def clean_text(text):
    # 1. Fix encoding issues like mojibake or smart quotes
    text = ftfy.fix_text(text)

    # 2. Normalize ligatures and full-width characters
    text = unicodedata.normalize("NFKC", text)

    # 3. Remove non-printable control characters (ASCII 0–31 and 127)
    text = re.sub(r'[\x00-\x1F\x7F]', '', text)

    # 4. Remove unnecessary backslashes before quotes (like d\'Océanographie → d'Océanographie)
    text = re.sub(r'\\([\'"“”‘’`´])', r'\1', text)

    # 5. Add space between glued lowercase-uppercase boundaries (e.g., datasetEuropean → dataset European)
    text = re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', text)

    # 6. Insert space before URLs (e.g., doihttps://... → doi https://...)
    text = re.sub(r'(?<=[a-zA-Z])(?=https?://)', ' ', text)

    # 7. Replace all weird dash characters with standard hyphen
    text = re.sub(r'[\u2010\u2011\u2012\u2013\u2014\u2212]', '-', text)

    # 8. Add space after punctuation if not followed by space
    text = re.sub(r'([,;!?)\]\}])(?=\S)', r'\1 ', text)


    # 9. Flatten all whitespace (spaces, tabs, newlines) into a single space
    text = re.sub(r'\s+', ' ', text)

    # 10. Remove any leftover leading/trailing space
    return text.strip()

In [17]:


def extract_text_from_xml(xml_path):
    parser = etree.XMLParser(recover=True)
    tree = etree.parse(xml_path, parser)
    root = tree.getroot()
    text_parts = []
    for elem in root.iter():
        if elem.text and elem.text.strip():
            text_parts.append(elem.text.strip())
        if elem.tail and elem.tail.strip():
            text_parts.append(elem.tail.strip())
    
    text =  ''.join(text_parts)
    return clean_text(text)

In [18]:
textx = extract_text_from_xml('data/train/XML/10.3389_fmicb.2024.1456637.xml')
textx[:10000]

'-//NLM//DTD Journal Publishing DTD v2.3 20070202//ENpublicjournalpublishing.dtd2.3nlm2jats3.xsl1open_accesspmc Front Microbiol Front Microbiol Front. Microbiol.Frontiers in Microbiology1664-302XFrontiers Media S.A.1142099310.3389/fmicb.2024.1456637Microbiology Original Research Conditional expression of flagellar motility, curli fimbriae, and biofilms in Shiga toxin- producing Escherichia albertii Carter Michelle Qiu1*Carychao Diana1Lindsey Rebecca L.21Produce Safety and Microbiology Research Unit, U.S. Department of Agriculture, Agricultural Research Service, Western Regional Research Center, Albany, CA, United States2Enteric Diseases Laboratory Branch, Centers for Disease Control and Prevention, Atlanta, GA, United States Edited by: Renmao "Tim" Tian, Illinois Institute of Technology, United States Reviewed by: Steven L. Foley, National Center for Toxicological Research (FDA) , United States Salvatore Walter Papasergi, National Research Council (CNR) , Italy*Correspondence: Michelle

In [19]:
import fitz
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    doc.close()
    return clean_text(text)

In [20]:
textp = extract_text_from_pdf('data/train/PDF/10.3389_fmicb.2024.1456637.pdf')
textp[:10000]

'TYPE Original Research PUBLISHED 09 September 2024DOI 10.3389/fmicb.2024.1456637OPEN ACCESSEDITED BYRenmao "Tim" Tian, Illinois Institute of Technology, United States REVIEWED BYSteven L. Foley, National Center for Toxicological Research(FDA) , United States Salvatore Walter Papasergi, National Research Council (CNR) , Italy*CORRESPONDENCEMichelle Qiu Cartermichelle.carter@usda.gov RECEIVED 28 June 2024ACCEPTED 14 August 2024PUBLISHED 09 September 2024CITATIONCarter MQ, Carychao D and Lindsey RL(2024) Conditional expression of flagellarmotility, curli fimbriae, and biofilms in Shigatoxin- producing Escherichia albertii.Front. Microbiol. 15:1456637.doi: 10.3389/fmicb.2024.1456637COPYRIGHT© 2024 Carter, Carychao and Lindsey. This isan open-access article distributed under theterms of the Creative Commons Attribution License (CC BY) . The use, distribution orreproduction in other forums is permitted, provided the original author(s) and thecopyright owner(s) are credited and that theorigi

In [28]:
import re

def extract_context(text, pattern, window=20):
    """
    Extract context around regex matches from given text.

    Parameters:
    - text (str): Input document text.
    - pattern (str): Regex pattern to match.
    - window (int): Number of words before and after match to include in context.

    Returns:
    - List of dicts with 'match' and 'context' keys.
    """

    matches = []

    # Normalize weird dash characters to simple hyphen
    text = re.sub(r'[\u2010\u2011\u2012\u2013\u2014\u2212]', '-', text)

    # Tokenize the full text once
    tokens = text.split()

    # Map character index → token index
    char_to_token = {}
    index = 0
    for i, tok in enumerate(tokens):
        index = text.find(tok, index)
        for j in range(index, index + len(tok)):
            char_to_token[j] = i
        index += len(tok)

    # Find matches using re.finditer()
    for match in re.finditer(pattern, text):
        raw_match = match.group()
        cleaned_match = re.sub(r'[\s\-]', '', raw_match)  # Remove spaces/dashes inside ID

        start_char = match.start()

        if start_char in char_to_token:
            token_index = char_to_token[start_char]
            start = max(0, token_index - window)
            end = min(len(tokens), token_index + window + 1)
            context_window = tokens[start:end]

            matches.append({
                'match': cleaned_match,
                'context': ' '.join(context_window)
            })

    return matches

In [29]:
extract_context(textp, r'SAMN[\s\-]?\d{8}')

[{'match': 'SAMN12620691',
  'context': 'ofisolation Gen Bank Bio Sample Numbera Swimming Motilitystx2f -positive E. albertii LB5% TSBPond water with 10%pigeon droppings RM9973American crow (Corvusbrachyrhynchos) /2009SAMN12620691-++RM9974American crow (Corvusbrachyrhynchos) /2009SAMN12620692---RM9976American crow (Corvusbrachyrhynchos) /2009SAMN12620693-++RM10507Brown-headed cowbird(Molothrus ater) /2009SAMN12620694+++RM10705Brown-headed cowbird(Molothrus ater) /2009SAMN12620697+++RM15112Oregon Junco (Juncohyemalis) /2011SAMN12620700---RM15113Oregon Junco (Juncohyemalis) /2011SAMN12620701---RM15114Oregon Junco (Juncohyemalis)'},
 {'match': 'SAMN12620692',
  'context': 'Bio Sample Numbera Swimming Motilitystx2f -positive E. albertii LB5% TSBPond water with 10%pigeon droppings RM9973American crow (Corvusbrachyrhynchos) /2009SAMN12620691-++RM9974American crow (Corvusbrachyrhynchos) /2009SAMN12620692---RM9976American crow (Corvusbrachyrhynchos) /2009SAMN12620693-++RM10507Brown-headed cowb

In [ ]:
samns = re.findall(r'SAMN[\s\-]?\d{8}', textx)
print(len(samns))
print(samns)

26
['SAMN12620691', 'SAMN12620692', 'SAMN12620693', 'SAMN12620694', 'SAMN12620697', 'SAMN12620700', 'SAMN12620701', 'SAMN12620702', 'SAMN12620703', 'SAMN12620704', 'SAMN03019926', 'SAMN04498560', 'SAMN04505646', 'SAMN04505647', 'SAMN07159041', 'SAMN08199278', 'SAMN07159045', 'SAMN09534374', 'SAMN07159044', 'SAMN08172567', 'SAMN33094111', 'SAMN33094114', 'SAMN33094099', 'SAMN33094102', 'SAMN33094112', 'SAMN17525956']


In [ ]:
samns = re.findall(r'SAMN[\s\-]?\d{8}', textp)
print(len(samns))
print(samns)

26
['SAMN12620691', 'SAMN12620692', 'SAMN12620693', 'SAMN12620694', 'SAMN12620697', 'SAMN12620700', 'SAMN12620701', 'SAMN12620702', 'SAMN12620703', 'SAMN12620704', 'SAMN03019926', 'SAMN04498560', 'SAMN04505646', 'SAMN04505647', 'SAMN07159041', 'SAMN08199278', 'SAMN07159045', 'SAMN09534374', 'SAMN07159044', 'SAMN08172567', 'SAMN33094111', 'SAMN33094114', 'SAMN33094099', 'SAMN33094102', 'SAMN33094112', 'SAMN17525956']


In [59]:
import sys
print(sys.executable)

/Users/aswinbenny/Downloads/projects/make_data_count/dataset-reference-finder/.venv/bin/python
